# 제품 review 데이터의 감성 분석

***

### 목표 - 감성 분석의 기본 이해: 감성 분석이 무엇이며 그 응용에 대해 학습

- 데이터
    - (홀수) amazon_uk_shoes_products_dataset_2021_12.csv 
- 데이터 전처리: 머신 러닝 작업을 위해 데이터 전 처리하는 경험을 쌓음.
    - 데이터 정리 (노이즈 제거, 결측값 처리 등).
    - 텍스트 토큰화 및 불용어 제거.
    - 텍스트 정규화 (스테밍 또는 표제어 추출).
- 모델 구현: 수업에서 활용한 모델을 활용해서 구현.
    - 수업에서 홯용한 모델을 적용.
- 모델 학습 및 튜닝
    - 데이터셋을 학습 및 테스트 세트로 분할.
    - 모델 학습 및 하이퍼파라미터 튜닝 (예: 그리드 서치 또는 랜덤 서치 사용).
    - 교차 검증과 같은 기술을 사용하여 견고성 보장.
- 모델 평가: 적절한 지표를 사용하여 모델의 성능을 평가.
    - 정확도, 정밀도, 재현율, F1 점수 및 ROC-AUC와 같은 지표를 사용하여 모델 평가.
    - 오버피팅과 언더피팅을 논의하고 이를 해결하는 방법 학습.
- 시각화 및 해석: 결과를 시각화하고 해석
    - Matplotlib 또는 Seaborn과 같은 라이브러리를 사용하여 결과 시각화.
    - 혼동 행렬 및 ROC 곡선 생성.

***

## 데이터 전처리

### 데이터 확인
- url : 상품 URL 주소
- product_name : 상품명
- reviewer_name : 리뷰 작성자 이름
- review_title : 리뷰 제목
- review_text : 리뷰 내용
- review_rating : 리뷰 별점
- verified_purchase : 구매 확인 여부
- review_date : 리뷰 작성 날짜
- helpful_count : 리뷰가 도움이 된 사람의 수
- uniq_id : 리뷰 ID
- scraped_at : 리뷰를 가져온 시점

### 데이터 정리
- 결측치 확인
- 실제 구매자가 작성하지 않은 리뷰 제외
- 영어 리뷰만 사용

In [16]:
# 필요한 라이브러리 설치 및 다운
%pip install langdetect

from langdetect import detect
import re

import pandas as pd

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 데이터 불러오기
file_path = './amazon_uk_shoes_products_dataset_2021_12.csv'
df = pd.read_csv(file_path)

In [3]:
# 결측치 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6823 entries, 0 to 6822
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   url                6823 non-null   object 
 1   product_name       6823 non-null   object 
 2   reviewer_name      6823 non-null   object 
 3   review_title       6822 non-null   object 
 4   review_text        6814 non-null   object 
 5   review_rating      6823 non-null   float64
 6   verified_purchase  6823 non-null   bool   
 7   review_date        6823 non-null   object 
 8   helpful_count      1953 non-null   object 
 9   uniq_id            6823 non-null   object 
 10  scraped_at         6823 non-null   object 
dtypes: bool(1), float64(1), object(9)
memory usage: 539.8+ KB


In [4]:
# 결측치 제거
df = df.dropna(subset=['review_title','review_text']) #review_title과 review_text열에 있는 결측치에 대해서만 전체 행 제거
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6813 entries, 0 to 6822
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   url                6813 non-null   object 
 1   product_name       6813 non-null   object 
 2   reviewer_name      6813 non-null   object 
 3   review_title       6813 non-null   object 
 4   review_text        6813 non-null   object 
 5   review_rating      6813 non-null   float64
 6   verified_purchase  6813 non-null   bool   
 7   review_date        6813 non-null   object 
 8   helpful_count      1950 non-null   object 
 9   uniq_id            6813 non-null   object 
 10  scraped_at         6813 non-null   object 
dtypes: bool(1), float64(1), object(9)
memory usage: 592.1+ KB


In [8]:
df['verified_purchase'] = df['verified_purchase'].astype(str) # 문자열 변환

# 실제 구매자가 작성하지 않은 리뷰 제외
df = df[df.verified_purchase != 'False']
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6800 entries, 0 to 6822
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   url                6800 non-null   object 
 1   product_name       6800 non-null   object 
 2   reviewer_name      6800 non-null   object 
 3   review_title       6800 non-null   object 
 4   review_text        6800 non-null   object 
 5   review_rating      6800 non-null   float64
 6   verified_purchase  6800 non-null   object 
 7   review_date        6800 non-null   object 
 8   helpful_count      1950 non-null   object 
 9   uniq_id            6800 non-null   object 
 10  scraped_at         6800 non-null   object 
dtypes: float64(1), object(10)
memory usage: 637.5+ KB


In [19]:
# 감성분석에 필요한 'review_title', 'review_text', 'review_rating' 열만 선택
df = df[['review_rating', 'review_title', 'review_text']]

In [14]:
# 영어로 된 리뷰만 남기기

# 'review_text'가 영어인 행만 남기고 나머지 제거 하는 함수 정의
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

df = df[df['review_text'].apply(is_english)]
df.info()

In [17]:
# 텍스트 전처리 함수
def preprocess_text(text):
    # 소문자 변환
    text = text.lower()
    # 숫자 제거
    text = re.sub(r'\d+', '', text)
    # 연속된 공백을 하나의 공백으로 통일
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# 'review_title'과 'review_text' 전처리
df['review_title'] = df['review_title'].apply(preprocess_text)
df['review_text'] = df['review_text'].apply(preprocess_text)

In [20]:
# 데이터 확인
df

,review_rating,review_title,review_text
0,5.0,love em,love these. was looking for converses and thes...
1,2.0,the plastic ripped,"the shoes are very cute, but after the nd day ..."
2,5.0,good quality,good quality
3,5.0,good,great
14,5.0,perfect right outta the box,true to size. if between i'd probably go with ...
...,...,...,...
6813,5.0,great for early walkers,the only shoes (after many tries) that worked ...
6814,3.0,three stars,too narrow hard to get on for a toddler
6815,5.0,said they were very comfortable.,my son loves them. said they were very comfort...
6816,2.0,they are smaller than other shoes the same size,size but they are smaller than the size my son...


In [21]:
# 필요 라이브러리 다운

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# NLTK 데이터 다운로드 (처음 한 번만 실행)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### 데이터 토큰화

In [22]:
# 텍스트 토큰화
df['review_title_token'] = df['review_title'].apply(word_tokenize)
df['review_text_token'] = df['review_text'].apply(word_tokenize)

/tmp/ipykernel_14813/406563728.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_title_token'] = df['review_title'].apply(word_tokenize)
/tmp/ipykernel_14813/406563728.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_text_token'] = df['review_text'].apply(word_tokenize)


In [23]:
df

,review_rating,review_title,review_text,review_title_token,review_text_token
0,5.0,love em,love these. was looking for converses and thes...,"[love, em]","[love, these, ., was, looking, for, converses,..."
1,2.0,the plastic ripped,"the shoes are very cute, but after the nd day ...","[the, plastic, ripped]","[the, shoes, are, very, cute, ,, but, after, t..."
2,5.0,good quality,good quality,"[good, quality]","[good, quality]"
3,5.0,good,great,[good],[great]
14,5.0,perfect right outta the box,true to size. if between i'd probably go with ...,"[perfect, right, outta, the, box]","[true, to, size, ., if, between, i, 'd, probab..."
...,...,...,...,...,...
6813,5.0,great for early walkers,the only shoes (after many tries) that worked ...,"[great, for, early, walkers]","[the, only, shoes, (, after, many, tries, ), t..."
6814,3.0,three stars,too narrow hard to get on for a toddler,"[three, stars]","[too, narrow, hard, to, get, on, for, a, toddler]"
6815,5.0,said they were very comfortable.,my son loves them. said they were very comfort...,"[said, they, were, very, comfortable, .]","[my, son, loves, them, ., said, they, were, ve..."
6816,2.0,they are smaller than other shoes the same size,size but they are smaller than the size my son...,"[they, are, smaller, than, other, shoes, the, ...","[size, but, they, are, smaller, than, the, siz..."


### 불용어 제거

In [24]:
# 불용어 제거 함수
stop_words = set(stopwords.words('english'))

def remove_stopwords(tokens):
    return [word for word in tokens if word.lower() not in stop_words]

# 불용어 제거 적용
df['review_title_token'] = df['review_title_token'].apply(remove_stopwords)
df['review_text_token'] = df['review_text_token'].apply(remove_stopwords)

/tmp/ipykernel_14813/2704780323.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_title_token'] = df['review_title_token'].apply(remove_stopwords)
/tmp/ipykernel_14813/2704780323.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_text_token'] = df['review_text_token'].apply(remove_stopwords)


In [25]:
df

,review_rating,review_title,review_text,review_title_token,review_text_token
0,5.0,love em,love these. was looking for converses and thes...,"[love, em]","[love, ., looking, converses, half, price, uni..."
1,2.0,the plastic ripped,"the shoes are very cute, but after the nd day ...","[plastic, ripped]","[shoes, cute, ,, nd, day, wearing, tongue, sta..."
2,5.0,good quality,good quality,"[good, quality]","[good, quality]"
3,5.0,good,great,[good],[great]
14,5.0,perfect right outta the box,true to size. if between i'd probably go with ...,"[perfect, right, outta, box]","[true, size, ., 'd, probably, go, lower, end, ..."
...,...,...,...,...,...
6813,5.0,great for early walkers,the only shoes (after many tries) that worked ...,"[great, early, walkers]","[shoes, (, many, tries, ), worked, early, walk..."
6814,3.0,three stars,too narrow hard to get on for a toddler,"[three, stars]","[narrow, hard, get, toddler]"
6815,5.0,said they were very comfortable.,my son loves them. said they were very comfort...,"[said, comfortable, .]","[son, loves, ., said, comfortable, .]"
6816,2.0,they are smaller than other shoes the same size,size but they are smaller than the size my son...,"[smaller, shoes, size]","[size, smaller, size, son, outgrowing, ., disa..."


### 데이터 정규화

In [26]:
# 표제어 추출 함수
lemmatizer = WordNetLemmatizer()

def lemmas(tokens):
    return [lemmatizer.lemmatize(word, pos='v') for word in tokens] # 'v'는 동사(verb)라는 뜻

# 표제어 추출 적용
df['review_title_token'] = df['review_title_token'].apply(lemmas)
df['review_text_token'] = df['review_text_token'].apply(lemmas)

/tmp/ipykernel_14813/978887466.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_title_token'] = df['review_title_token'].apply(lemmas)
/tmp/ipykernel_14813/978887466.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_text_token'] = df['review_text_token'].apply(lemmas)


In [27]:
df

,review_rating,review_title,review_text,review_title_token,review_text_token
0,5.0,love em,love these. was looking for converses and thes...,"[love, em]","[love, ., look, converse, half, price, unique—..."
1,2.0,the plastic ripped,"the shoes are very cute, but after the nd day ...","[plastic, rip]","[shoe, cute, ,, nd, day, wear, tongue, start, ..."
2,5.0,good quality,good quality,"[good, quality]","[good, quality]"
3,5.0,good,great,[good],[great]
14,5.0,perfect right outta the box,true to size. if between i'd probably go with ...,"[perfect, right, outta, box]","[true, size, ., 'd, probably, go, lower, end, ..."
...,...,...,...,...,...
6813,5.0,great for early walkers,the only shoes (after many tries) that worked ...,"[great, early, walkers]","[shoe, (, many, try, ), work, early, walker, b..."
6814,3.0,three stars,too narrow hard to get on for a toddler,"[three, star]","[narrow, hard, get, toddler]"
6815,5.0,said they were very comfortable.,my son loves them. said they were very comfort...,"[say, comfortable, .]","[son, love, ., say, comfortable, .]"
6816,2.0,they are smaller than other shoes the same size,size but they are smaller than the size my son...,"[smaller, shoe, size]","[size, smaller, size, son, outgrow, ., disappo..."
